참고

1. keybert와 kiwi 형태소분석기를 사용하여 키워드추출
- https://datainclude.me/posts/Keybert%EC%99%80_kiwi%ED%98%95%ED%83%9C%EC%86%8C%EB%B6%84%EC%84%9D%EA%B8%B0%EB%A5%BC_%EC%82%AC%EC%9A%A9%ED%95%98%EC%97%AC_%ED%82%A4%EC%9B%8C%EB%93%9C%EC%B6%94%EC%B6%9C_%ED%95%98%EA%B8%B0/

2. kiwi document
- https://bab2min.github.io/kiwipiepy/v0.8.0/kr/
- https://mindscale.kr/course/text-mining/kiwi/


## Import Library

In [19]:
import pandas as pd
import numpy as np
from konlpy.tag import *
from kiwipiepy import Kiwi
kiwi = Kiwi()

from keybert import KeyBERT
kw_model = KeyBERT()
from gensim import models
from sklearn.metrics.pairwise import cosine_similarity

ko_model = models.fasttext.load_facebook_model('model/cc.ko.300.bin/cc.ko.300.bin')

## Load Data

In [2]:
sample = pd.read_csv('data/sample_description.csv', encoding="CP949")

sample["message"] = sample["message"].str.replace("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]"," ",regex=True)
sample["description"] = sample["description"].str.replace("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]"," ",regex=True)

In [3]:
sample

,user_id,message,description,reason,category
0,1,나만의 길을 찾을 때까지 인생이라는 게임의 레버를 당기는 법,좋아하는 일로 행복하게 일하는 상위 밀레니얼 프리워커 드로우앤드류입니다,자율성과 자유,경제적 독립
1,2,UI UX 아티클 백과사전 국내 해외 매일 업로드,년차 UX 컨설턴트이면서 년차 UX 강사로서 하루도 빠지지 않고 수집...,NaN,NaN
2,3,UXUI 취업 이직 스터디 포트폴리오 컨설팅,이런분들께 아주 도움되실꺼에요 UX디자이너가 되기위해 취업준비중인 취준생 ...,NaN,NaN
3,4,데일리 디자인 정보 공유 세션,매일 디자인 소식 하나를 픽해서 공유합니다 실무에 유용한 정보를 체계적으로 정리해...,NaN,NaN
4,5,피그마 학습방 서비스 기획자와 함께해요,피그마가 궁금하신가요 피그마를 쓰고 있으신가요 피그마를 사용하며 프로덕트를 만드...,NaN,NaN
5,6,UX Writer로 성장해볼까요,UX Writer 직무가 궁금하신 분들 현업에서는 어떻게 일하는지 궁금하신 분들 ...,NaN,NaN
6,7,BX 디자이너 취업 이직하기,신입 주니어 BX 브랜드 디자이너 분들의 구직을 위한 포트폴리오 코칭과 멘토링을...,NaN,NaN
7,8,성장하는 프로덕트 디자이너 SaaS 환영,SaaS와 프로덕트 디자인에 대한 정보와 의견을 나눠요 하루에 하나라도 제대로 읽...,NaN,NaN
8,9,디자이너로서 성공하고 싶은 사람들이 모인 방,안녕하세요 저는 고졸 국비지원 과정을 통해 스타트업 디자이너로 커리어를 시작하여...,NaN,NaN
9,10,프로토파이 Protopie 프로토타이핑 도움방,프로토타이핑 도구 중 high fidelity 높은 재현 을 가진 Protopie에...,NaN,NaN


## NLP 1-1.형태소 분석

In [7]:
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            results.append(token)
    return results

In [14]:
for i in range(0, len(sample)):
    print("\n Sample Description {index}.  \n".format(index=i))
    print("raw data: ", sample["description"][i])
    print("명사 추출: ", noun_extractor(sample["description"][i]))


 Sample 0.  

raw data:  좋아하는 일로 행복하게 일하는 상위    밀레니얼 프리워커 드로우앤드류입니다 
명사 추출:  ['행복', '상위', '밀레니얼', '프리', '워커', '드로우', '앤드']

 Sample 1.  

raw data:        년차 UX 컨설턴트이면서   년차 UX 강사로서 하루도 빠지지 않고 수집했던 방대한 양의 자료들을 공개하고자 합니다      매일 다양한 주제의 국내 해외 UI UX 아티클이 업로드됩니다      국내 해외 UI UX 트렌드를 빠르게 파악할 수 있습니다      구글 브런치 서핏 등에 검색하지 않고도 해시태그를 통해 아티클을 손쉽게 찾을 수 있습니다      지금 바로 입장하실 수 있습니다 
명사 추출:  ['년차', 'UX', '컨설턴트', '년차', 'UX', '강사', '하루', '수집', '자료', '공개', '다양', '주제', '국내', '해외', 'UI', 'UX', '아티클이', '업로드', '국내', '해외', 'UI', 'UX', '트렌드', '파악', '구글', '브런치', '서핏', '검색', '해시태그', '아티', '입장']

 Sample 2.  

raw data:    이런분들께 아주 도움되실꺼에요   UX디자이너가 되기위해 취업준비중인 취준생   타 전공자분들을 위한 UX입문과정UX팀으로 이직을 위한 이직 준비 과정
명사 추출:  ['도움', 'UX', '디자이너', '취업', '준비', '취준생', '전공', 'UX', '입문', '과정', 'UX', '이직', '이직', '준비', '과정']

 Sample 3.  

raw data:  매일 디자인 소식 하나를 픽해서 공유합니다  실무에 유용한 정보를 체계적으로 정리해 드립니다  뼈가 되고 살이 되는 정보로 튼튼한 디자이너로 성장하세요 
명사 추출:  ['디자인', '소식', '하나', '공유', '실무', '유용', '정보', '체계', '정리', '정보', '디자이너', '성장']

 Sample

In [16]:
for i in range(0, len(sample)):
    print("\n Sample Message {index}.  \n".format(index=i))
    print("raw data: ", sample["message"][i])
    print("명사 추출: ", noun_extractor(sample["message"][i]))


 Sample Message 0.  

raw data:  나만의 길을 찾을 때까지 인생이라는 게임의 레버를 당기는 법
명사 추출:  ['인생', '게임', '레버']

 Sample Message 1.  

raw data:  UI UX 아티클 백과사전  국내 해외    매일 업로드
명사 추출:  ['UI', 'UX', '아티클', '백과사전', '국내', '해외', '업로드']

 Sample Message 2.  

raw data:  UXUI 취업 이직 스터디 포트폴리오 컨설팅
명사 추출:  ['UXUI', '취업', '이직', '스터디', '포트폴리오', '컨설팅']

 Sample Message 3.  

raw data:  데일리 디자인 정보 공유 세션
명사 추출:  ['데일리', '디자인', '정보', '공유', '세션']

 Sample Message 4.  

raw data:  피그마 학습방   서비스 기획자와 함께해요 
명사 추출:  ['피그마', '학습', '서비스', '기획자']

 Sample Message 5.  

raw data:  UX Writer로 성장해볼까요 
명사 추출:  ['UX', 'Writer', '성장']

 Sample Message 6.  

raw data:  BX 디자이너  취업 이직하기
명사 추출:  ['BX', '디자이너', '취업', '이직']

 Sample Message 7.  

raw data:  성장하는 프로덕트 디자이너    SaaS 환영
명사 추출:  ['성장', '프로덕트', '디자이너', 'SaaS', '환영']

 Sample Message 8.  

raw data:    디자이너로서 성공하고 싶은 사람들이 모인 방
명사 추출:  ['디자이너', '성공', '사람']

 Sample Message 9.  

raw data:  프로토파이 Protopie  프로토타이핑 도움방
명사 추출:  ['프로토파이', 'Protopie', '프로', '타이핑', '도움']

 Sample Me

## NLP 1-2. 키워드 추출 + 형태소분석

In [22]:
for i in range(0, len(sample)):
    print("\n Sample Description {index}.  \n".format(index=i))
    print("raw data: ", sample["description"][i])
    
    case4 = noun_extractor(sample["description"][i])
    case4 = ' '.join(case4)
    
    print("명사추출: ", case4)
    print("명사 + 키워드 추출: ", kw_model.extract_keywords(case4, keyphrase_ngram_range=(1, 1), top_n = 5))


 Sample Description 0.  

raw data:  좋아하는 일로 행복하게 일하는 상위    밀레니얼 프리워커 드로우앤드류입니다 
명사추출:  행복 상위 밀레니얼 프리 워커 드로우 앤드
명사 + 키워드 추출:  [('밀레니얼', 0.6389), ('드로우', 0.5647), ('워커', 0.5525), ('행복', 0.5066), ('앤드', 0.4906)]

 Sample Description 1.  

raw data:        년차 UX 컨설턴트이면서   년차 UX 강사로서 하루도 빠지지 않고 수집했던 방대한 양의 자료들을 공개하고자 합니다      매일 다양한 주제의 국내 해외 UI UX 아티클이 업로드됩니다      국내 해외 UI UX 트렌드를 빠르게 파악할 수 있습니다      구글 브런치 서핏 등에 검색하지 않고도 해시태그를 통해 아티클을 손쉽게 찾을 수 있습니다      지금 바로 입장하실 수 있습니다 
명사추출:  년차 UX 컨설턴트 년차 UX 강사 하루 수집 자료 공개 다양 주제 국내 해외 UI UX 아티클이 업로드 국내 해외 UI UX 트렌드 파악 구글 브런치 서핏 검색 해시태그 아티 입장
명사 + 키워드 추출:  [('컨설턴트', 0.4987), ('아티클이', 0.4641), ('ui', 0.407), ('입장', 0.3833), ('브런치', 0.3785)]

 Sample Description 2.  

raw data:    이런분들께 아주 도움되실꺼에요   UX디자이너가 되기위해 취업준비중인 취준생   타 전공자분들을 위한 UX입문과정UX팀으로 이직을 위한 이직 준비 과정
명사추출:  도움 UX 디자이너 취업 준비 취준생 전공 UX 입문 과정 UX 이직 이직 준비 과정
명사 + 키워드 추출:  [('ux', 0.4419), ('전공', 0.3267), ('과정', 0.2891), ('이직', 0.2849), ('입문', 0.271)]

 Sample Description 3.  

raw data:  매일 디

명사 + 키워드 추출:  [('여행', 0.5109), ('일상', 0.4748), ('공유', 0.4591), ('와인', 0.3741), ('음식', 0.3721)]

 Sample Description 26.  

raw data:    오늘 듣고 있는 플레이리스트  음악 취향을 공유해요   부담감 없이 가볍게 오고가는 클럽입니다   콘서트메이트를 구하거나 바이닐 LP 스토어 오프라인 팟을 구하는 것또한 OK 
명사추출:  플레이 리스트 음악 취향 공유 부담감 클럽 콘서트 메이트 바이닐 LP 스토어 오프라인 OK
명사 + 키워드 추출:  [('콘서트', 0.4241), ('공유', 0.3853), ('클럽', 0.3683), ('메이트', 0.3573), ('오프라인', 0.3432)]

 Sample Description 27.  

raw data:  대학에서 진행했던 워크숍으로  참가자분들과 함께 시나리오 및 캐릭터 분석을 트레이닝합니다 
명사추출:  대학 진행 워크숍 참가자 시나리오 캐릭터 분석 트레이닝
명사 + 키워드 추출:  [('워크숍', 0.5183), ('캐릭터', 0.4635), ('트레이닝', 0.4503), ('진행', 0.3756), ('분석', 0.3342)]

 Sample Description 28.  

raw data:  매 주 회고를 통해 나를 되돌아보고  더 나은 내가 되려는 사람들이 모인 클럽입니다  일주일에 한 번씩 회고를 진행합니다  금   일요일 사이에 자유롭게 진행해주세요    
명사추출:  회고 사람 클럽 주일 회고 진행 일요일 사이 자유 진행
명사 + 키워드 추출:  [('진행', 0.5082), ('일요일', 0.4979), ('클럽', 0.4452), ('사이', 0.4004), ('자유', 0.3814)]

 Sample Description 29.  

raw data:  인상깊은 문장을 꾹꾹 눌러 담아 써봐요  문장들을 함께 나누고  일상을 깊고 단단하게 만들어가요 
명사추출:  인상 문장 문장 일

## NLP 2. word embedding (fasttext)

In [106]:
word2vec = []
for i in range(0, len(sample)):        
    case4 = noun_extractor(sample["description"][i])
    case4 = ' '.join(case4)
    
    sample_word = [x[0] for x in kw_model.extract_keywords(case4, keyphrase_ngram_range=(1, 1), top_n = 5)]
    print(sample_word)
    sample_vec = np.mean(ko_model.wv[sample_word],axis=0)
    
    word2vec.append(sample_vec)

['밀레니얼', '드로우', '워커', '행복', '앤드']
['컨설턴트', '아티클이', 'ui', '입장', '브런치']
['ux', '전공', '과정', '이직', '입문']
['성장', '공유', '유용', '정보', '디자인']
['성장', '공유', '프로덕트', '공부', '정보']
['writer', 'ux', '성장', '현업', '직무']
['bx', '멘토링', '포트폴리오', '신입', '진행']
['saas', '성장', '정보', '하루', '디자인']
['스타트업', '성공', '영역', '실패', '창업']
['protopie', 'fidelity', '인터랙', '방법', '구현']
['디지털', '유저', '디자인', '중심', '자유']
['포트폴리오', '그래픽', '경험', '디자인', '이직']
['saas', 'cx', '경영진', '네트워킹', '메이커']
['마케터', '세상', '변화', '기술', '고객']
['manager', 'owner', 'product', 'po', '그거']
['참여', '담소', '사람', '모두', '업무']
['마케팅', '디지털', '공유', '모집', '스터디']
['gtm', 'ga', '진행', '스터디', '온라인']
['공부', '세상', '도움', '브랜드']
['네트워킹', '스타트업', '성장', '커피', '오프라인']
['멘토링', '파생상품', 'job', '운용사', '세일즈']
['클럽', '금융', '노력', '상품', '투자']
['클럽', '커리어', '트레이딩', '퀀트', '시스템']
['ai', '미디어아트', '아이디어', '가상현실', '미술']
['인터레스팅', '콘텐츠', '선정', '공유', 'esg']
['여행', '일상', '공유', '와인', '음식']
['콘서트', '공유', '클럽', '메이트', '오프라인']
['워크숍', '캐릭터', '트레이닝', '진행', '분석']
['진행', '일요일', '클럽', '사이', '자유']

In [122]:
result = pd.concat([sample,pd.DataFrame(word2vec)], axis=1)
result.head()

,user_id,message,description,reason,category,0,1,2,3,4,...,290,291,292,293,294,295,296,297,298,299
0,1,나만의 길을 찾을 때까지 인생이라는 게임의 레버를 당기는 법,좋아하는 일로 행복하게 일하는 상위 밀레니얼 프리워커 드로우앤드류입니다,자율성과 자유,경제적 독립,-0.047428,0.015121,-0.167326,0.030501,-0.088213,...,-0.035906,-0.013809,0.040263,-0.009675,0.074850,0.005420,0.002102,0.033656,0.079658,0.060873
1,2,UI UX 아티클 백과사전 국내 해외 매일 업로드,년차 UX 컨설턴트이면서 년차 UX 강사로서 하루도 빠지지 않고 수집...,NaN,NaN,0.052307,-0.043585,-0.117675,0.014773,0.002465,...,-0.033788,-0.099364,-0.065231,-0.018093,0.007475,-0.007121,0.026936,-0.002956,0.041100,0.024266
2,3,UXUI 취업 이직 스터디 포트폴리오 컨설팅,이런분들께 아주 도움되실꺼에요 UX디자이너가 되기위해 취업준비중인 취준생 ...,NaN,NaN,-0.100198,-0.157610,-0.239711,-0.112113,-0.021419,...,0.004315,-0.011044,-0.038786,0.008448,0.002081,-0.005875,-0.082463,0.103412,0.101734,0.102226
3,4,데일리 디자인 정보 공유 세션,매일 디자인 소식 하나를 픽해서 공유합니다 실무에 유용한 정보를 체계적으로 정리해...,NaN,NaN,-0.014615,-0.005615,-0.155185,-0.050214,0.008901,...,-0.015111,0.016107,0.050471,-0.043690,0.071938,-0.015405,-0.035267,0.150049,-0.001377,0.066158
4,5,피그마 학습방 서비스 기획자와 함께해요,피그마가 궁금하신가요 피그마를 쓰고 있으신가요 피그마를 사용하며 프로덕트를 만드...,NaN,NaN,0.059302,-0.062709,-0.187960,-0.013293,-0.060740,...,0.048967,0.010665,0.055120,-0.055475,0.089833,0.008196,-0.034343,0.092580,0.052726,0.017212


In [147]:
def largest_number(arr, rank):
    unique_nums = set(arr)
    sorted_nums = sorted(unique_nums, reverse=True)
    return sorted_nums[rank]

In [148]:
for i in range(0,len(sample)):
    similar_top1 = np.where(cosine_similarity(word2vec)[i] == largest_number(cosine_similarity(word2vec)[i], 1))[0][0]
    similar_top2 = np.where(cosine_similarity(word2vec)[i] == largest_number(cosine_similarity(word2vec)[i], 2))[0][0]
    similar_top3 = np.where(cosine_similarity(word2vec)[i] == largest_number(cosine_similarity(word2vec)[i], 3))[0][0]
    
    print("\n Sample Description {index}.  \n".format(index=i))
    print("raw data: ", sample["description"][i])
    print("similar top1: ", sample["description"][similar_top1])
    print("similar top2: ", sample["description"][similar_top2])
    print("similar top3: ", sample["description"][similar_top3])


 Sample Description 0.  

raw data:  좋아하는 일로 행복하게 일하는 상위    밀레니얼 프리워커 드로우앤드류입니다 
similar top1:  미라클모닝과 함께 기록하는 습관 만들기   오늘 하고 싶은 할 일  책 제목 나누고 싶은 구절  어제 하루에 대한 짧은 흔적 회고  기상 후 생각과 오늘의 마음 가짐 매일 나누어요 
similar top2:  인사를 처음 시작하는 초심자들과 나날이 깊어지고 넓어져 가는 우리 일들에 대해서 함께 공부하고  공유하고 서로 도움을 주고 받는 클럽입니다    데이터 분석  ESG 함께해요  D
similar top3:  맛있는 음식을 먹을 때  축하할 때  여행할 때 와인은 향과 스토리를 가져다 주고 사람은 추억과 기록을 남깁니다  일상 속 와인을 공유하고 자랑하고 이야기도 나누어 보아요 

 Sample Description 1.  

raw data:        년차 UX 컨설턴트이면서   년차 UX 강사로서 하루도 빠지지 않고 수집했던 방대한 양의 자료들을 공개하고자 합니다      매일 다양한 주제의 국내 해외 UI UX 아티클이 업로드됩니다      국내 해외 UI UX 트렌드를 빠르게 파악할 수 있습니다      구글 브런치 서핏 등에 검색하지 않고도 해시태그를 통해 아티클을 손쉽게 찾을 수 있습니다      지금 바로 입장하실 수 있습니다 
similar top1:  이미 Production에서 사용중인 개발자와 새로 공부하는 뉴비까지  안드로이드의 새로운 UI 프레임워크에 대한 최신 정보를 얻으세요 
similar top2:     간단한 소개와 함께 입장해주세요    주제 분야에 관계 없이 재밌는 소식 정보에 대해 자유롭게 이야기 나눕니다     재밌는 소식이나 정보가 있으면  톡톡 공유해주세요 
similar top3:  신입 주니어 BX  브랜드  디자이너 분들의 구직을 위한 포트폴리오 코칭과 멘토링을 진행합니다

 Sample Description 2.  

raw data